# Build IMDb actor dataframe

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import csv
from ast import literal_eval
from project_utils import *
from pandas.io.json import json_normalize
from functools import reduce

%reload_ext lab_black

In [2]:
credits_df = pd.read_csv("./data/tmdb_5000_credits.csv", sep=",", quotechar='"')
credits_df.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [3]:
movies_df = pd.read_csv("./data/tmdb_5000_movies.csv", sep=",", quotechar='"').rename(
    {"id": "movie_id"}, axis=1
)
movies_df.head()

,budget,genres,homepage,movie_id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
movies_df = pd.read_csv("./data/tmdb_5000_movies.csv", sep=",", quotechar='"').rename(
    {"id": "movie_id"}, axis=1
)
movies_df.head()

,budget,genres,homepage,movie_id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


## Process movies DataFrame
Flatten and clean movies DataFrame

In [5]:
pr_movies_df = movies_df.copy()
pr_movies_df[
    (pr_movies_df["budget"] != 0)
    & (pr_movies_df["revenue"] != 0)
    & (pr_movies_df["popularity"] != 0)
    & (pr_movies_df["runtime"] != 0)
    & (pr_movies_df["vote_average"] != 0)
    & (pr_movies_df["vote_count"] != 0)
]
# Transform json to list of dictionaries within cols
movies_df_json_cols = [
    "genres",
    "production_companies",
    "keywords",
    "production_countries",
    "spoken_languages",
]
cols_to_drop = ["homepage", "overview", "tagline", "original_title"]
key = "name"
movie_id_col = "movie_id"
pr_movies_df = col_json_to_dict(pr_movies_df, movies_df_json_cols)

# Transform list of dictionaries to sets containing a relevant value from dictionaries
for col in movies_df_json_cols:
    pr_movies_df = col_dict_to_set(pr_movies_df, col, key)
pr_movies_df = pr_movies_df.drop(columns=cols_to_drop)
pr_movies_df.head()

,budget,genres,movie_id,keywords,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count
0,237000000,"{Action, Adventure, Science Fiction, Fantasy}",19995,"{society, future, 3d, culture clash, anti war,...",en,150.437577,"{Twentieth Century Fox Film Corporation, Ingen...","{United Kingdom, United States of America}",2009-12-10,2787965087,162.0,"{English, Español}",Released,Avatar,7.2,11800
1,300000000,"{Fantasy, Adventure, Action}",285,"{love of one's life, ocean, east india trading...",en,139.082615,"{Second Mate Productions, Jerry Bruckheimer Fi...",{United States of America},2007-05-19,961000000,169.0,{English},Released,Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"{Action, Adventure, Crime}",206647,"{sequel, united kingdom, based on novel, spy, ...",en,107.376788,"{Columbia Pictures, Danjaq, B24}","{United Kingdom, United States of America}",2015-10-26,880674609,148.0,"{Français, Deutsch, Español, English, Italiano}",Released,Spectre,6.3,4466
3,250000000,"{Action, Thriller, Drama, Crime}",49026,"{crime fighter, vigilante, hostage drama, cove...",en,112.312950,"{Syncopy, Legendary Pictures, Warner Bros., DC...",{United States of America},2012-07-16,1084939099,165.0,{English},Released,The Dark Knight Rises,7.6,9106
4,260000000,"{Action, Adventure, Science Fiction}",49529,"{mars civilization, princess, medallion, alien...",en,43.926995,{Walt Disney Pictures},{United States of America},2012-03-07,284139100,132.0,{English},Released,John Carter,6.1,2124


## Process credits DataFrame
Flatten and clean credits DataFrame

In [6]:
pr_credits_df = credits_df.copy()

# Parse json cols to list of dictionaries
credits_df_json_cols = ["cast", "crew"]
key = "name"
pr_credits_df = col_json_to_dict(pr_credits_df, credits_df_json_cols)
# Create the actors col
pr_credits_df = pr_credits_df.assign(actors=pr_credits_df[credits_df_json_cols[0]])
# Get sets of relevant values from the lists of dictionaries
for col in credits_df_json_cols:
    if col != "crew":
        pr_credits_df = col_dict_to_set(pr_credits_df, col, key)

In [7]:
# Filter crew dictionaries with Director as job
crew_col = "crew"
job_field = "job"
values = ["Director"]
pr_credits_df = col_filter_dict_with_vals(
    pr_credits_df, crew_col, job_field, values
).rename({crew_col: values[0]})
pr_credits_df = col_dict_to_set(pr_credits_df, crew_col, key)

In [8]:
# Select a specific number of actors as nodes
actors_col = "actors"
order_key = "order"
# Set the actor importances (the order key. The main actor is order 0) to consider for taking an actor as a node
values = [0]
get_value = lambda dict_: dict_.get(key)
actors_series = col_filter_dict_with_vals(pr_credits_df, actors_col, order_key, values)[
    actors_col
].explode()
actors_series = actors_series[actors_series.notna()]
actors_list = list(set(actors_series.apply(get_value).to_list()))
print("The list of actors contains {:2d} entries".format(len(actors_list)))
print(actors_list[:5])

The list of actors contains 1971 entries
['Nick Nolte', 'Kayla Jackson', 'Simon Pegg', 'Gene Wilder', 'Guo Tao']


In [9]:
name_key = "name"
values = actors_list
pr_credits_df = col_filter_dict_with_vals(pr_credits_df, actors_col, name_key, values)
pr_credits_df = pr_credits_df.explode(actors_col)
pr_credits_df = pr_credits_df[pr_credits_df[actors_col].notna()]
actors_series = pr_credits_df[actors_col]
pr_credits_df = pr_credits_df.assign(
    **{actors_col: pr_credits_df[actors_col].apply(get_value)}
)
pr_credits_df

,movie_id,title,cast,crew,actors
0,19995,Avatar,"{Wes Studi, Colin Bleasdale, Matthew Chamberla...",{James Cameron},Sam Worthington
0,19995,Avatar,"{Wes Studi, Colin Bleasdale, Matthew Chamberla...",{James Cameron},Zoe Saldana
0,19995,Avatar,"{Wes Studi, Colin Bleasdale, Matthew Chamberla...",{James Cameron},Sigourney Weaver
0,19995,Avatar,"{Wes Studi, Colin Bleasdale, Matthew Chamberla...",{James Cameron},Stephen Lang
0,19995,Avatar,"{Wes Studi, Colin Bleasdale, Matthew Chamberla...",{James Cameron},Giovanni Ribisi
...,...,...,...,...,...
4801,126186,Shanghai Calling,"{Alan Ruck, Daniel Henney, Bill Paxton, Zhu Sh...",{Daniel Hsia},Bill Paxton
4802,25975,My Date with Drew,"{Drew Barrymore, Matt LeBlanc, Bill D'Elia, Er...","{Jon Gunn, Brian Herzlinger, Brett Winn}",Drew Barrymore
4802,25975,My Date with Drew,"{Drew Barrymore, Matt LeBlanc, Bill D'Elia, Er...","{Jon Gunn, Brian Herzlinger, Brett Winn}",Eric Roberts
4802,25975,My Date with Drew,"{Drew Barrymore, Matt LeBlanc, Bill D'Elia, Er...","{Jon Gunn, Brian Herzlinger, Brett Winn}",Samuel L. Jackson


In [10]:
# Remove actors taken as nodes from cast column
actor_cast_col = "actor_cast"
pr_credits_df = pr_credits_df.reset_index(drop=True)
remove_from_cast = lambda row: row[credits_df_json_cols[0]] - set({row[actors_col]})
pr_credits_df = pr_credits_df.assign(
    **{credits_df_json_cols[0]: pr_credits_df.apply(remove_from_cast, axis=1)}
).drop(columns="title")
pr_credits_df.head()

,movie_id,cast,crew,actors
0,19995,"{Richard Whiteside, Wes Studi, Hanniyah Muhamm...",{James Cameron},Sam Worthington
1,19995,"{Richard Whiteside, Wes Studi, Hanniyah Muhamm...",{James Cameron},Zoe Saldana
2,19995,"{Richard Whiteside, Wes Studi, Hanniyah Muhamm...",{James Cameron},Sigourney Weaver
3,19995,"{Richard Whiteside, Wes Studi, Hanniyah Muhamm...",{James Cameron},Stephen Lang
4,19995,"{Richard Whiteside, Wes Studi, Hanniyah Muhamm...",{James Cameron},Giovanni Ribisi


## Build actors index DataFrame
Get actors index and most relevant features

In [11]:
actor_cols = ["name", "gender", "order"]
actors_idx_df = json_normalize(actors_series)[actor_cols]
actors_idx_df = (
    actors_idx_df.rename(columns={actor_cols[0]: actors_col})
    .groupby(actors_col)
    .aggregate({actor_cols[1]: max, actor_cols[2]: list})
    .reset_index()
    .reset_index()
    .rename(columns={"index": "actor_id"})
)
actors_idx_df.head()

,actor_id,actors,gender,order
0,0,"""Weird Al"" Yankovic",0,"[18, 13, 13, 0]"
1,1,50 Cent,2,"[4, 2, 4, 0, 4, 16, 7, 6]"
2,2,AJ Michalka,1,"[13, 9, 18, 0]"
3,3,Aamir Khan,2,"[0, 0]"
4,4,Aaron Abrams,2,"[34, 19, 0, 11]"


In [12]:
actors_idx_df.tail()

,actor_id,actors,gender,order
1966,1966,Zoe Kazan,1,"[9, 10, 3, 4, 16, 18, 3, 0]"
1967,1967,Zoe Lister-Jones,1,"[13, 0]"
1968,1968,Zoe Saldana,1,"[1, 2, 4, 1, 6, 7, 7, 2, 9, 0, 2, 6, 4, 1, 1, ..."
1969,1969,Zoe Sloane,0,[0]
1970,1970,Zooey Deschanel,1,"[2, 1, 5, 1, 2, 3, 1, 4, 2, 6, 4, 3, 3, 2, 2, ..."


## Build actors_index-credits DataFrame
Add actor features to the credits DataFrame

In [13]:
actors_idx_credits_df = pr_credits_df.merge(
    actors_idx_df, right_on=actors_col, left_on=actors_col, how="inner"
)
actors_idx_credits_df.head(20)

,movie_id,cast,crew,actors,actor_id,gender,order
0,19995,"{Richard Whiteside, Wes Studi, Hanniyah Muhamm...",{James Cameron},Sam Worthington,1653,2,"[0, 1, 0, 0, 19, 4, 15, 0, 1, 4]"
1,534,"{Common, Bryce Dallas Howard, Chris Ashworth, ...",{McG},Sam Worthington,1653,2,"[0, 1, 0, 0, 19, 4, 15, 0, 1, 4]"
2,57165,"{Sinéad Cusack, Alejandro Naranjo, Matt Milne,...",{Jonathan Liebesman},Sam Worthington,1653,2,"[0, 1, 0, 0, 19, 4, 15, 0, 1, 4]"
3,18823,"{Pete Postlethwaite, Natalia Vodianova, Luke E...",{Louis Leterrier},Sam Worthington,1653,2,"[0, 1, 0, 0, 19, 4, 15, 0, 1, 4]"
4,10592,"{Rory Cochrane, Colin Farrell, Bruce Willis, J...",{Gregory Hoblit},Sam Worthington,1653,2,"[0, 1, 0, 0, 19, 4, 15, 0, 1, 4]"
5,253412,"{Josh Brolin, Demetri Goritsas, Vijay Lama, To...",{Baltasar Kormákur},Sam Worthington,1653,2,"[0, 1, 0, 0, 19, 4, 15, 0, 1, 4]"
6,13922,"{Clayne Crawford, Steve Harman, Logan Marshall...",{John Dahl},Sam Worthington,1653,2,"[0, 1, 0, 0, 19, 4, 15, 0, 1, 4]"
7,49527,"{James Thomas Bligh, Michael Laurence, Jonatha...",{Asger Leth},Sam Worthington,1653,2,"[0, 1, 0, 0, 19, 4, 15, 0, 1, 4]"
8,144336,"{Laurence Chavez, Michael Monks, Alan Gilmer, ...",{David Ayer},Sam Worthington,1653,2,"[0, 1, 0, 0, 19, 4, 15, 0, 1, 4]"
9,48289,"{Ciarán Hinds, Jesper Christensen, Helen Mirre...",{John Madden},Sam Worthington,1653,2,"[0, 1, 0, 0, 19, 4, 15, 0, 1, 4]"


## Build movies index DataFrame
Re-index movies 

In [14]:
movie_id_col = "movie_id"
title_col = "title"
new_movie_id_col = "new_movie_id"
movies_idx_df = (
    pr_movies_df[[movie_id_col, title_col]]
    .drop_duplicates()
    .reset_index()
    .rename(columns={"index": new_movie_id_col})
)
movies_idx_df.head()

,new_movie_id,movie_id,title
0,0,19995,Avatar
1,1,285,Pirates of the Caribbean: At World's End
2,2,206647,Spectre
3,3,49026,The Dark Knight Rises
4,4,49529,John Carter


In [15]:
movies_idx_csv_df = movies_idx_df.set_index(new_movie_id_col)
movies_idx_csv_df.to_csv("movies_idx.csv")

## Build actors-movies-credits DataFrame
Get actors from each movie and append their respective credits

In [16]:
actors_movies_credits_df = actors_idx_credits_df.merge(
    pr_movies_df, right_on=movie_id_col, left_on=movie_id_col, how="inner"
)
actors_movies_credits_df = (
    actors_movies_credits_df.merge(
        movies_idx_df.drop(columns=title_col),
        right_on=movie_id_col,
        left_on=movie_id_col,
        how="inner",
    )
    .drop(columns=movie_id_col)
    .rename(columns={new_movie_id_col: movie_id_col})
)
actors_movies_credits_df.head()

,cast,crew,actors,actor_id,gender,order,budget,genres,keywords,original_language,...,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count,movie_id
0,"{Richard Whiteside, Wes Studi, Hanniyah Muhamm...",{James Cameron},Sam Worthington,1653,2,"[0, 1, 0, 0, 19, 4, 15, 0, 1, 4]",237000000,"{Action, Adventure, Science Fiction, Fantasy}","{society, future, 3d, culture clash, anti war,...",en,...,"{United Kingdom, United States of America}",2009-12-10,2787965087,162.0,"{English, Español}",Released,Avatar,7.2,11800,0
1,"{Richard Whiteside, Wes Studi, Hanniyah Muhamm...",{James Cameron},Zoe Saldana,1968,1,"[1, 2, 4, 1, 6, 7, 7, 2, 9, 0, 2, 6, 4, 1, 1, ...",237000000,"{Action, Adventure, Science Fiction, Fantasy}","{society, future, 3d, culture clash, anti war,...",en,...,"{United Kingdom, United States of America}",2009-12-10,2787965087,162.0,"{English, Español}",Released,Avatar,7.2,11800,0
2,"{Richard Whiteside, Wes Studi, Hanniyah Muhamm...",{James Cameron},Sigourney Weaver,1734,1,"[2, 29, 27, 6, 4, 5, 0, 0, 6, 16, 1, 3, 12, 0,...",237000000,"{Action, Adventure, Science Fiction, Fantasy}","{society, future, 3d, culture clash, anti war,...",en,...,"{United Kingdom, United States of America}",2009-12-10,2787965087,162.0,"{English, Español}",Released,Avatar,7.2,11800,0
3,"{Richard Whiteside, Wes Studi, Hanniyah Muhamm...",{James Cameron},Stephen Lang,1762,2,"[3, 9, 0, 6, 3, 4, 9, 4, 5, 8, 0]",237000000,"{Action, Adventure, Science Fiction, Fantasy}","{society, future, 3d, culture clash, anti war,...",en,...,"{United Kingdom, United States of America}",2009-12-10,2787965087,162.0,"{English, Español}",Released,Avatar,7.2,11800,0
4,"{Richard Whiteside, Wes Studi, Hanniyah Muhamm...",{James Cameron},Giovanni Ribisi,647,2,"[5, 2, 2, 8, 5, 2, 2, 4, 12, 5, 4, 3, 5, 3, 6,...",237000000,"{Action, Adventure, Science Fiction, Fantasy}","{society, future, 3d, culture clash, anti war,...",en,...,"{United Kingdom, United States of America}",2009-12-10,2787965087,162.0,"{English, Español}",Released,Avatar,7.2,11800,0


## Build actors aggregated DataFrame

In [17]:
get_union = lambda s: reduce(set.union, s)
actor_id_col = "actor_id"
actors_agg_df = (
    actors_movies_credits_df.groupby(actors_col, as_index=False)
    .aggregate(
        {
            "movie_id": set,
            "cast": get_union,
            "crew": get_union,
            "actor_id": max,
            "gender": max,
            "budget": "mean",
            "genres": get_union,
            "keywords": get_union,
            "original_language": set,
            "popularity": "mean",
            "production_companies": get_union,
            "production_countries": get_union,
            "release_date": list,
            "revenue": "mean",
            "runtime": sum,
            "spoken_languages": get_union,
            "status": list,
            "title": set,
            "vote_average": "mean",
            "vote_count": "mean",
        }
    )
    .set_index(actor_id_col)
)
actors_agg_df.head()

,actors,movie_id,cast,crew,gender,budget,genres,keywords,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count
actor_id,,,,,,,,,,,,,,,,,,,,
0,"""Weird Al"" Yankovic","{1585, 2338, 3619, 2551}","{Nicky Whelan, Danielle Harris, Chris Hardwick...","{Rick Friedberg, Rob Zombie, Peter Segal, Jay ...",0,17000000.0,"{Action, Horror, Comedy, Crime}","{music spoof, duringcreditsstinger, satire, pe...",{en},14.215532,"{Spectacle Entertainment Group, Orion Pictures...",{United States of America},"[1994-03-18, 2009-08-28, 1996-05-24, 1989-07-21]",2.263852e+07,366.0,{English},"[Released, Released, Released, Released]","{The Naked Gun 33⅓: The Final Insult, UHF, Spy...",5.750,256.000
1,50 Cent,"{2400, 609, 2275, 1712, 1232, 1233, 761, 1597}","{Kevin Kline, Brendan Jeffers, Alec Rayme, Mph...","{Jon Turteltaub, Jon Avnet, Brian A Miller, An...",2,36652500.0,"{Action, Thriller, Crime, Comedy, Romance, Drama}","{hunting, muslim, drug, ship, escape, work eth...",{en},30.167837,"{InVenture Entertainment, Court Five, Emmett/F...",{United States of America},"[2013-10-31, 2010-01-12, 2008-09-11, 2013-10-0...",9.993918e+07,861.0,"{English, Deutsch, Український, العربية, Pусск...","[Released, Released, Released, Released, Relea...","{The Frozen Ground, Righteous Kill, Morning Gl...",6.175,772.625
2,AJ Michalka,"{4203, 908, 1381, 327}","{Richard Fullerton, Gabriel Basso, Jacob Rhode...","{J.J. Abrams, Brad J. Silverman, Peter Jackson...",1,37500000.0,"{Fantasy, Thriller, Music, Mystery, Science Fi...","{loss of daughter, teenage love, duringcredits...",{en},19.503003,"{Fast Track Productions, DreamWorks SKG, New Z...","{United Kingdom, New Zealand, United States of...","[2010-08-20, 2009-12-26, 2011-06-08, 2013-10-04]",1.034682e+08,473.0,{English},"[Released, Released, Released, Released]","{The Lovely Bones, Super 8, Grace Unplugged, S...",6.500,906.750
3,Aamir Khan,"{3543, 1023}","{Kitu Gidwani, Maia Sethna, Navtej Singh Johar...","{Rakeysh Omprakash Mehra, Deepa Mehta}",2,1100000.0,{Drama},"{1940s, student, students' movement, war of in...",{hi},3.226925,"{Flicks Motion Pictures, UTV Motion Pictures, ...","{India, Canada}","[1998-09-16, 2006-01-26]",6.015562e+06,258.0,{हिन्दी},"[Released, Released]","{Earth, Rang De Basanti}",6.900,53.000
4,Aaron Abrams,"{956, 2242, 2267, 380}","{Attila Sebesy, Joan Moore, Thomas Kretschmann...","{Marc Abraham, Alexander Witt, Alejandro Amená...",2,33250000.0,"{Horror, Thriller, Mystery, Science Fiction, R...","{world cup, zombie, rescue, boxer, comeback, m...",{en},21.136737,"{Miramax Films, Intermittent Productions, Stri...","{Germany, France, Canada, United Kingdom, Spai...","[2015-10-01, 2005-06-02, 2008-09-07, 2004-09-10]",6.390146e+07,463.0,{English},"[Released, Released, Released, Released]","{Resident Evil: Apocalypse, Regression, Cinder...",6.250,631.750


In [18]:
actors_agg_df.to_pickle("actors_agg_df.pkl")

## Tests

In [19]:
# Cast should be set
actors_agg_df = pd.read_pickle("actors_agg_df.pkl")
type(actors_agg_df.loc[0, "cast"])

set

In [20]:
actors_agg_df.columns

Index(['actors', 'movie_id', 'cast', 'crew', 'gender', 'budget', 'genres',
       'keywords', 'original_language', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'title', 'vote_average', 'vote_count'],
      dtype='object')

In [21]:
actors_agg_df[["actors", "title"]]

,actors,title
actor_id,,
0,"""Weird Al"" Yankovic","{The Naked Gun 33⅓: The Final Insult, UHF, Spy..."
1,50 Cent,"{The Frozen Ground, Righteous Kill, Morning Gl..."
2,AJ Michalka,"{The Lovely Bones, Super 8, Grace Unplugged, S..."
3,Aamir Khan,"{Earth, Rang De Basanti}"
4,Aaron Abrams,"{Resident Evil: Apocalypse, Regression, Cinder..."
...,...,...
1966,Zoe Kazan,"{Me and Orson Welles, The Exploding Girl, Meek..."
1967,Zoe Lister-Jones,"{Breaking Upwards, Salt}"
1968,Zoe Saldana,"{The Book of Life, Guess Who, The Terminal, Gu..."


In [22]:
actor_entry = actors_agg_df.iloc[1968]
print("The actor is: " + actor_entry["actors"])
print("His/her movies are: " + str(actors_agg_df.iloc[1968]["title"]))

The actor is: Zoe Saldana
His/her movies are: {'The Book of Life', 'Guess Who', 'The Terminal', 'Guardians of the Galaxy', 'The Losers', 'Colombiana', 'Drumline', 'The Words', 'Crossroads', 'Avatar', 'Center Stage', 'Star Trek', 'Get Over It', 'Star Trek Beyond', 'Vantage Point', 'Blood Ties', 'Pirates of the Caribbean: The Curse of the Black Pearl', 'Star Trek Into Darkness', 'Takers', 'Out of the Furnace'}
